In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store, fetch_predictions
import pandas as pd
from datetime import datetime, timedelta, timezone


def fetch_days_data(days):
    current_date = pd.to_datetime(datetime.now(timezone.utc))
    fetch_data_from = current_date - timedelta(days=(365+days)) 
    fetch_data_to = current_date - timedelta(days=365)
    print(fetch_data_from, fetch_data_to)
    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    # query = query.filter((fg.pickup_hour >= fetch_data_from))
    df = query.read()
    cond = (df["pickup_hour"] >= fetch_data_from) & (df["pickup_hour"] <= fetch_data_to)
    return df[cond]

In [4]:
ts_data = fetch_days_data(180)

2023-09-06 15:55:32.675314+00:00 2024-03-04 15:55:32.675314+00:00
2025-03-04 10:55:32,677 INFO: Initializing external client
2025-03-04 10:55:32,679 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 10:55:33,948 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215681
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.00s) 


In [5]:
from src.data_utils import transform_ts_data_info_features_and_target

features, targets = transform_ts_data_info_features_and_target(ts_data, window_size=24*28, step_size=23)

ValueError: No data could be transformed. Check if input DataFrame is empty or window size is too large.

In [18]:
from src.pipeline_utils import get_pipeline
pipeline = get_pipeline()
pipeline.fit(features, targets)

2025-03-04 10:56:13,020 WARNING: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead

2025-03-04 10:56:13,021 WARNING: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead



NameError: name 'features' is not defined

In [ ]:
from sklearn.metrics import mean_absolute_error
predictions = pipeline.predict(features)

In [ ]:
test_mae = mean_absolute_error(targets, predictions)
print(f"{test_mae:.4f}")

In [ ]:
from src.inference import load_metrics_from_registry 

metric = load_metrics_from_registry()

In [ ]:
metric

In [ ]:
from src.inference import load_model_from_registry
model = load_model_from_registry()

In [ ]:
import joblib  
import src.config
# Save the pipeline  
joblib.dump(pipeline, config.MODELS_DIR / "lgb_model.pkl")

In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(features)
output_schema = Schema(targets)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [ ]:
model_registry = project.get_model_registry()
model_registry

In [ ]:
modelv2 = model_registry.sklearn.create_model(
    name="taxi_demand_predictor_next_hour",
    metrics={"test_mae": test_mae},
    description="LightGBM regressor V2",
    input_example=features.sample(),
    model_schema=model_schema,
)

In [24]:
modelv2.save('E:\ms fall 2024\sping 2025\MLOPS\sp25_taxi-main\models\\lgb_model.pkl')

2025-03-04 10:56:13,822 WARNING: SyntaxWarning: invalid escape sequence '\m'

2025-03-04 10:56:13,824 WARNING: SyntaxWarning: invalid escape sequence '\m'

2025-03-04 10:56:13,958 WARNING: SyntaxWarning: invalid escape sequence '\m'

2025-03-04 10:56:13,965 WARNING: SyntaxWarning: invalid escape sequence '\m'

2025-03-04 10:56:13,966 WARNING: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead

2025-03-04 10:56:13,967 WARNING: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead



NameError: name 'modelv2' is not defined

In [ ]:
from src.inference import load_model_from_registry
model = load_model_from_registry(-1)

In [ ]:
models = model_registry.get_models(name=config.MODEL_NAME)

In [ ]:
models

In [29]:
max(models, key=lambda model: model.version)


2025-03-04 10:56:14,743 WARNING: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead

2025-03-04 10:56:14,743 WARNING: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead



NameError: name 'models' is not defined

In [ ]:
load_metrics_from_registry()